# PROGRAM 4 : Understand the impact of different optimizers on the training process.
# Implement various optimizers (e.g., SGD, Adam, RMSprop).
# Train a neural network on the CIFAR-10 dataset using each optimizer.
# Compare the convergence speed and final accuracy of each optimizer.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the CIFAR-10 CSV dataset
def load_cifar10_csv(filepath):
    df = pd.read_csv(filepath)
    labels = df['label'].values
    images = df.drop('label', axis=1).values.reshape(-1, 32, 32, 3) / 255.0
    return images, labels

# Load training and test data from csv files
X, y = load_cifar10_csv('cifar10.csv') 

# Convert labels to categorical
y = tf.keras.utils.to_categorical(y, 10)

# Split data into training and validation sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def build_and_train_model(optimizer, epochs=10):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=1, validation_data=(X_test, y_test))
    return history

optimizers = {'SGD': SGD(), 'Adam': Adam(), 'RMSprop': RMSprop()}
histories = {name: build_and_train_model(optimizer) for name, optimizer in optimizers.items()}

# Visualize training performance
plt.figure(figsize=(12, 8))
for name in optimizers.keys():
    plt.plot(histories[name].history['accuracy'], label=f'{name} train')
    plt.plot(histories[name].history['val_accuracy'], label=f'{name} val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy for Different Optimizers (TensorFlow/Keras)')
plt.show()

# Print final accuracy
for name, history in histories.items():
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    print(f"{name} - Train Accuracy: {final_train_acc:.4f}, Validation Accuracy: {final_val_acc:.4f}")